# E-Commerce: Product Range Analysis 

**The purpose of the project:**

- **To learn the range of products, what products does the store have to offer and how are they sold?**


- **What are the popular products and what are not.**


- **Which products yield the greatest profit.**


- **Which products are usually sold together.**

### The basis of the analysis should be the fact that the recommendations are aimed to store manager so that he can change or improve the range of products.

# Decomposition:

## - EDA 
- **Checking for missing and duplicate values, adjusting the data types to facilitate the analysis process, general information about our data.**

## - Analyze the product range

- **We will check distribution by products, what sells more and what less, and in addition we will categorize the products to check if there is a leading category and if so what it is and how to possibly expand the products in that category, or perhaps reduce products in an unpopular category to free up resources for products in another popular category.**



- **We will also make a distribution by date, we will check time patterns and try to find out what are the times when sales increase and where they actually decrease.**


## - Formulate and test statistical hypotheses

-  **Will expanding the range of products in a category that is considered popular result in an increase in sales?**

- **Should unprofitable products be dropped?**

## - Conclusion and Recomendation


##   Downloading data

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from IPython.display import display
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from plotly import graph_objects as go
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
import sidetable
import scipy.stats as stats
        
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize 
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter

In [15]:
data = pd.read_csv('C:\\Users\\evyke\\OneDrive\\Desktop\\projects\\Final_Project\\ecommerce_dataset_us.csv', sep='\t')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,11/29/2018 08:26,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,11/29/2018 08:26,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,11/29/2018 08:26,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,11/29/2018 08:26,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,11/29/2018 08:26,3.39,17850.0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 28.9+ MB


In [ ]:
data.info()

In [ ]:
#function to calculate missing values and duplicates 
def check(df):
    print('_____________________________________')
    print('MISSING VALUES:')
    print(df.isna().sum())
    print('______________________________________')
    print('SHARE OF MISSING VALUES (%):')
    print(round(((df.isnull().sum()/len(df))*100),2))
    print('______________________________________')
    print('DUPLICATES:')
    print(df.duplicated().sum()) 

In [ ]:
check(data)

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.describe()

In [ ]:
data.InvoiceDate.describe()

In [ ]:
data.nunique()

## data overview:
 
- the mean and the standart deviaton(std) in quantity are so far from each other, which is mean we have some big orders contains  a large amount of items and probably impact directly affects the price, so the same phenomenon happens there as well.

- we have 25900 orders and 4372 customers.

- our range time is 2018-11-29 / 2019-12-07(1y,9d). the most popular day is 2019-19-29. this data will be usefull for seasons investigation.

## missing and duplicates:

- 135080 values are missing the in CustomerID field (24.93% in total)

- 1454 values are missing in the Description field (0.27% in total)

now im going over the columns that need to be chacked.

# Preprocessing data and EDA

In [ ]:
data.sample(10)

In [ ]:
data[data['InvoiceNo'].str.startswith('C')]

##  Missing values in CustomerID column

In [ ]:
data[data['CustomerID'].isna()].sample(10)

I do not recognize any repeating pattern and therefore I cannot fill them with other truth values, on the other hand I will not delete them, for further analysis purposes.
How did it happen? There can be many technical reasons that the information is not saved well, or unregistered customers who have made purchases.

In [ ]:
data['CustomerID'] = data['CustomerID'].fillna('unknwon')

## UnitPrice columns

What can be a problem with this field are negative values and zero, let's check

In [ ]:
data[data['UnitPrice'] < 0]

According to what is detailed in the description, these are debt payments, and they have nothing to do with the purposes of our analysis, so we will delete them.

In [ ]:
data[data['UnitPrice']==0].sample(10)

You can see that these lines have something in common with the missing values in "Description".

In [ ]:
len(data[data['UnitPrice']==0])

negative values and zero prices affected on revenue, and our main goals is to analyze the rang product, so those rows should be removed.(2512 rows)

In [ ]:
data = data.query('UnitPrice > 0')

## Quantity column

In [ ]:
data.query('Quantity <= 0').sort_values(by='Quantity', ascending = True).head(20)

its seen tjat all negative quantity is return transaction, cuz all invoiceNo are with 'C' and thats stand for "correction".
any way, my opinion is thats rows are imported for understending why peploe returned them? so lets keep them and remove them from our datasets. 

In [ ]:
returned = data.query('Quantity <= 0')
len(returned)

In [ ]:
data = data.query('Quantity > 0')

# StockCode

Apparently, product code names should contain only numbers or letters and numbers together. Let's try to find the exceptions that contain only letters.

In [ ]:
sorted(data['StockCode'].unique().tolist(), reverse = True)[0:22]

These lines do not seem to be related to the sale or transaction more like operating expenses so I will delete them.

In [ ]:
irelevant_transaction = ['m',
 'gift_0001_50',
 'gift_0001_40',
 'gift_0001_30',
 'gift_0001_20',
 'gift_0001_10',
 'S',
 'POST',
 'PADS',
 'M',
 'DOT',
 'DCGSSGIRL',
 'DCGSSBOY',
 'DCGS0076',
 'DCGS0070',
 'DCGS0069',
 'DCGS0004',
 'DCGS0003',
 'C2',
 'BANK CHARGES',
 'B',
 'AMAZONFEE']

for word in irelevant_transaction:
    data = data[~data['StockCode'].str.contains(word)]

## Description 

In [ ]:
#cheacking missing value in this field
data['Description'].isna().sum()

At first we saw that there were missing values in this field (scroll up to the beginning of the notebook), now you can see that they have disappeared, and this is because in previous steps we deleted rows with negative or zero values and they disappeared along with them.

In [ ]:
#looking for desription sales in lowercase
data[data['Description'].str.islower()]

We also see the same principle regarding the descriptions that appeared in small letters that are no longer in the dataset, this is because they were removed together with the rows that had zero values in the price field because they were not rows that record a normal transaction but all kinds of returns or other expenses that are not a sale.

# adding fields

## Revenue  

In [ ]:
data['Revenue'] = data.Quantity * data.UnitPrice
data['Revenue'].sum()

In [ ]:
data.head()

## Month and Date

In [ ]:
data['Month'] = data['InvoiceDate'].dt.strftime('%Y-%m')
data['Date'] = data['InvoiceDate'].dt.floor('d')

data.head()

# Distribution and Outliers

In [ ]:
# This function will show us the extreme values from below and above.

def eda(column):
    lower_limit = np.percentile(data[column], 2)
    upper_limit = np.percentile(data[column], 98)
    print('Limits for column:', column)
    print()
    print('lower_limit:',lower_limit)
    print('upper_limit:',upper_limit)

## Quantity

In [ ]:
data.Quantity.describe()

In [ ]:
eda('Quantity')

Now we will explore the extreme values

In [ ]:
quant_out = data.query('Quantity < 1 or Quantity > 72')
quant_out.head()

In [ ]:
quant_out.sort_values(by='Quantity').head()

In [ ]:
quant_out.sort_values(by='Quantity',ascending = False).head(10)

In [ ]:
quant_out.CustomerID.nunique()/len(quant_out)

**Immediate conclusion**: 
there are no specific customers who make the big purchases. So there is nothing to invest in finding    the most profitable customers, there are too many of them.

In [ ]:
print('the precentage of quantity outliers from all dataset is: {:.2%}'.format(len(quant_out)/len(data)))

this is small precentage and could be removed, but lets see how much ther share from revenue.

In [ ]:
print('Quantity outliers % in Revenue: {:.2%}'.format(quant_out['Revenue'].sum()/data['Revenue'].sum()))

wow! its a big share. so i will not removed them.

In [ ]:
sns.histplot(data.query('1 < Quantity < 72')['Quantity'], bins=10)
plt.title('Quantity distribution without outliers')
plt.show()

## UnitPrice

In [ ]:
data.UnitPrice.describe()

In [ ]:
eda('UnitPrice')

In [ ]:
UnitPrice_out = data.query('UnitPrice < 0.39 or UnitPrice > 12.75')
UnitPrice_out.head()

In [ ]:
UnitPrice_out.sort_values(by ='UnitPrice').head(10)

In [ ]:
UnitPrice_out.sort_values(by ='UnitPrice',ascending=False).head(15)

In [ ]:
print('the precentage of UnitPrice outliers from all dataset is: {:.2%}'.format(len(UnitPrice_out)/len(data)))

In [ ]:
print('UnitPrice outliers % in Revenue: {:.2%}'.format(UnitPrice_out['Revenue'].sum()/data['Revenue'].sum()))

In [ ]:
sns.histplot(data.query('0.39 < UnitPrice < 12.75')['UnitPrice'], bins=10)
plt.title('UnitPrice distribution without outliers')
plt.show()

## Revenue

In [ ]:
data.Revenue.describe()

In [ ]:
eda('Revenue')

In [ ]:
sns.histplot(data.query('0.83 < Revenue < 112.5')['Revenue'], bins=10)
plt.title('Revenue Distrubition')
plt.show()

## preprocess summary

- We identified invoices that are actually refund/cancellation of transaction (C) and removed them along with the lines where a negative price and quantity appeared.


- Customers who did not have an ID number filled in 'unknwon'.


- We located the unusual code names of the products and removed them. 

- For example, commissions for sales websites, gift cards, etc.


- We added a profit column, date day - month - year, month.


- We presented the exceptions in the three numerical columns (income, quantity, price unit).

- And since the exceptions were a large percentage of the income, it was decided to keep them and not delete them.

- We removed about 8% rows from the original dataset.

# Categorization

## price range categories

In [ ]:
print("25% price range:",np.percentile(data['UnitPrice'], 25))
print("50% price range:",np.percentile(data['UnitPrice'], 50))
print("75% price range:",np.percentile(data['UnitPrice'], 75))
print("95% price range:",np.percentile(data['UnitPrice'], 95))

In [ ]:
price_ranges = {
    (0, 0): 'free',
    (0, 1.25): 'low',
    (1.25, 2.07): 'medium',
    (2.07, 4.13): 'high medium',
    (4.13, 9.94): 'high',
    (9.94, float('inf')): 'elite'
}

def price_cat(i):
    for (start, end), label in price_ranges.items():
        if start < i <= end:
            return label 


In [ ]:
data['Price_Range'] = data['UnitPrice'].apply(price_cat)
sns.histplot(data['Price_Range'],bins=10)
plt.title('Price_Range Distribution')
plt.show()

## RFM

Recency (R) - the time that has passed since the customer made the last purchase.

Frequency (F) - how many total purchases the customer made.

Monetary (M) - the total amount of the customer's purchases in the business.

In [ ]:
# removing uncompleted month to invoid negative values in the "recency" column.
data=data.query('InvoiceDate > "2018-12-01" and InvoiceDate < "2019-12-01"')

In [ ]:
data['Date'].max()

In [ ]:
last_date=dt.datetime(2019,11,30)
last_date

In [ ]:
rfm=data.groupby('Description').agg({
    'Date': lambda x: (last_date-x.max()).days,
    'InvoiceNo': 'nunique',
    'Revenue':'sum'
}).reset_index()
rfm.rename(columns={'Date':'recency',
                    'InvoiceNo':'frequency',
                    'Revenue':'monetary'},inplace=True)
rfm.head(10)

Now I'm going to make this metric simpler and more useful. The numbers from 1-4 symbolize the level of value, the frequency, and the most recently bought in descending order.

In [ ]:
rfm['R']=pd.qcut(rfm.recency,5,[1,2,3,4],duplicates = 'drop')
rfm['F']=pd.qcut(rfm.frequency,4,[4,3,2,1])
rfm['M']=pd.qcut(rfm.monetary,4,[4,3,2,1])
rfm.head()

dividing the data into segment.

In [ ]:
def rfm_name(row):
    
    if row['R'] == 1:
        r_name = 'recent'
    if row['R'] == 2:
        r_name = 'less recent'
    if row['R'] == 3:
        r_name = 'old'
    if row['R'] == 4:
        r_name = 'oldest'
    
    if row['F'] == 1:
        f_name = 'top seller'
    if row['F'] == 2:
        f_name = 'popular'
    if row['F'] == 3:
        f_name = 'unpopular'
    if row['F'] == 4:
        f_name = 'rare'
        
    if row['M'] == 1:
        m_name = 'best value'
    if row['M'] == 2:
        m_name = 'good value'
    if row['M'] == 3:
        m_name = 'low value'
    if row['M'] == 4:
        m_name = 'cheap'
    
    return r_name + " / " + f_name + " / " + m_name
 

In [ ]:
rfm['RFM_segment'] = rfm.apply(rfm_name, axis=1)
rfm.head()

now lets merge the rfm df with the source.

In [ ]:
data=data.merge(right=rfm[['Description','R','F','M','RFM_segment']], on='Description', how='left')
data.head()

In [ ]:
data.groupby('RFM_segment')['Revenue'].sum().reset_index().sort_values(by='Revenue',ascending=False)

The more frequently the product is found, the more it is sold and increases the income. In the last chapter of the project I described by drawing what the most profitable products are and which category is the most profitable.

## Division into categories

In [ ]:
Lem = WordNetLemmatizer()
queries = data['Description'].unique()

splitwords = [nltk.word_tokenize(str(query)) for query in queries]

word_list = []
for sublist in splitwords:
    for item in sublist:
        word_list.append(item)
        
        
lemmas = [Lem.lemmatize(w, pos = 'n') for w in word_list]

lemmas[:20]

In [ ]:
# Use the pos_tag() function to tag the words as nouns, verbs, etc.
lemmas = [str(x) for x in lemmas] #convert to srtings
lemmas = [x.lower() for x in lemmas] #convert to lowercase for nltk
tagged = nltk.pos_tag(lemmas)

# Use a list comprehension to select the nouns
nouns_word = [word for word, tag in tagged if tag == 'NN']
nouns_word[:20]

In [ ]:
len(nouns_word)

we got a large number of nouns, It would take a lot of precious time if I go through them one by one and try to assign them to a certain category, so I will take the 150 most common roots in the list and then it will make more sense to categorize them.

In [ ]:
#count each noun
counter_nouns = Counter(nouns_word)

#from list to dict, for sort.
top_nouns = pd.DataFrame.from_dict(counter_nouns, orient='index', columns=['count'])
top_nouns.reset_index(inplace=True)
top_nouns=top_nouns.sort_values(by='count', ascending=False).head(150)

#back to list
top_nouns = top_nouns['index'].to_list()
top_nouns

now i will clean the list.

In [ ]:
top_nouns_clean = [
 'bag',
 'box',
 'holder',
 'decoration',
 'necklace',
 'card',
 'bracelet',
 'cover',
 'mug',
 'candle',
 'mirror',
 'frame',
 'cream',
 'bowl',
 'clock',
 'charm',
 'jar',
 'hook',
 'bottle',
 'tray',
 'cup',
 'cutlery',
 'plate',
 'book',
 'tree',
 'trinket',
 'basket',
 'ring',
 'hanger',
 'doormat',
 'notebook',
 'bell',
 'warmer',
 'wallet',
 'picture',
 'parasol',
 'towel',
 'chain',
 'tissue',
 'doll',
 'jug',
 'stand',
 'sticker',
 'bin',
 'flag',
 'container']

In [ ]:
data['Description'] = data['Description'].str.lower()

In [ ]:
def match(x):
    for i, item in enumerate(top_nouns_clean):
        if item in x:
            return item
    return 'Other'

data['Category'] = data['Description'].apply(match)

data.head()

In [ ]:
data.stb.freq(['Category'])

now we can move on to sales anlysis.

<div class="alert alert-block alert-warning">
<b>Team Lead comment:</b> Analysis of categories??
    
    
<b>Student comment:</b> I combined all the analyzes and visualizations in another chapter.
</div>

## Sales Analysis

### top 10 ordered

In [ ]:
top_ordered = data['Description'].value_counts().head(10).reset_index()
top_ordered = top_ordered.rename(columns={'index' : 'item','Description':'num_purchases'})

top_ordered

In [ ]:
plt.figure(figsize=(15,5))

sns.barplot(x='num_purchases',y='item', data=top_ordered)

for i, v in enumerate(top_ordered.num_purchases):
    plt.text(v, i, str(round(v)), color='black')

plt.title("Top 10 items ordered",fontsize=20)
plt.xlabel('Count')
plt.show()

- The most purchased products are first of all products for home use and products related to food preparation and storage.

**Recommendation:** expand the range of products related to home accessories, home kitchen, food packaging solutions.

### Sales by Price_Range

In [ ]:
price_range_data = data.groupby('Price_Range').agg({'InvoiceNo':'count','Quantity':'sum','Revenue':'sum'}).reset_index()
price_range_data = price_range_data.rename(columns={'InvoiceNo':'num_purchases'})
price_range_data

In [ ]:
plt.figure(figsize=(15,5))

ax = sns.barplot(x='Price_Range',y='Quantity', data=price_range_data)

# adding the ratio number of each price range
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ratio = y / sum(price_range_data['Quantity'])  # Calculate the ratio
    ax.text(x, y, f'{ratio:.2%}', ha='center', va='bottom')


plt.title('Quantity Distribution by range price',fontsize=20)
plt.ylabel('Quantity(in milion)')
plt.show()

## conclusion:

- The results connect with reality, and as a rule you can see that as the price increases, there is a decrease in price (negative correlation).

### Sales by Category

In [ ]:
Category_Sales = data.groupby('Category').agg({'InvoiceNo':'count','Quantity':'sum','Revenue':'sum'}).reset_index()
Category_Sales = Category_Sales.rename(columns={'InvoiceNo':'num_purcheses'}).sort_values(by='num_purcheses',ascending=False)

Category_Sales

In [ ]:
plt.figure(figsize=(15,15))

sns.barplot(x='num_purcheses',y='Category', data=Category_Sales)

for i, v in enumerate(Category_Sales.num_purcheses):
    plt.text(v, i,str(round(v)), color='black')

plt.title("purcheses by Category",fontsize=25)
plt.xlabel('Count')
plt.ylabel('Category',size = 25)
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))

# Create the bar plot
ax = sns.barplot(x='Quantity', y='Category', data=Category_Sales.sort_values(by='Quantity', ascending=False))

# Add text annotations
for i, v in enumerate(Category_Sales.Quantity):
    plt.text(v, i, str(round(v)), color='black', ha='left', va='center')

plt.title("Quantity by Category", fontsize=25)
plt.xlabel('Count')
plt.ylabel('Category', size=25)
plt.show()


## Summary:

- The top five (identified) categories in the number of purchases are: bag,	box, holder, card, decoration. (descending order)


- The top five (identified) categories in the quantity are: bag, holder, box, card, decoration. (descending order)

## sales by months

In [ ]:
Sales_months = data.groupby('Month').agg({'InvoiceNo':'count','Quantity':'sum','Revenue':'sum'}).reset_index()
Sales_months = Sales_months.rename(columns={'InvoiceNo': 'num_purchases'})

Sales_months

In [ ]:
plt.figure(figsize=(15, 10))

# Create the bar plot
ax = sns.barplot(x='num_purchases', y='Month', data=Sales_months)

# Add text annotations
for i, v in enumerate(Sales_months.num_purchases):
    plt.text(v, i, str(round(v)), color='black', ha='left', va='center')

plt.title("purchases by Month", fontsize=25)
plt.xlabel('Count')
plt.ylabel('Month', size=25)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

# Create the bar plot
ax = sns.barplot(x='Quantity', y='Month', data=Sales_months)

# Add text annotations
for i, v in enumerate(Sales_months.Quantity):
    plt.text(v, i, str(round(v)), color='black', ha='left', va='center')

plt.title("Quantity by Month", fontsize=25)
plt.xlabel('Count', size=15)
plt.ylabel('Month', size=25)
plt.tight_layout()
plt.show()

- There is a strong relationship between the number of purchases and the amount of products, the months are almost identical in both histograms.

## Revenue Analysis

### top 10 profit items

In [ ]:
top_profit_item = data.groupby('Description')['Revenue'].sum().reset_index().sort_values(by='Revenue',ascending=False)[:10]
top_profit_item

In [ ]:
plt.figure(figsize=(15, 5),dpi= 80)

# Create the bar plot
ax = sns.barplot(x='Revenue', y='Description', data=top_profit_item)

# Add text annotations
for i, v in enumerate(top_profit_item.Revenue):
    plt.text(v, i, str(round(v)), color='black', ha='left', va='center')

plt.title("top 10 most profitable items", fontsize=25)
plt.xlabel('Count', size=15)
plt.ylabel('items', size=25)
plt.tight_layout()
plt.show()

- Note: These products do not represent final profit, because it is determined by the cost of the product and operating expenses, which is information that does not exist here

### Revenue by Price_Range

In [ ]:
plt.figure(figsize=(15,5))

ax = sns.barplot(x='Price_Range',y='Revenue', data=price_range_data)

# adding the ratio number of each price range
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ratio = y / sum(price_range_data['Revenue'])  # Calculate the ratio
    ax.text(x, y, f'{ratio:.2%}', ha='center', va='bottom')


plt.title('Revenue Distribution by range price',fontsize=20)
plt.ylabel('Revenue(in milion)')
plt.show()

- Note: Unlike the quantity and number of purchases, the highest income is actually from the " " category, which means that what is sold more does not necessarily bring the most income.

### Revenue by Category

In [ ]:
plt.figure(figsize=(15,10))

ax = sns.barplot(x='Revenue',y='Category', data=Category_Sales.sort_values(by='Revenue',ascending=False).head(10))

for i, v in enumerate(top_profit_item.Revenue):
    plt.text(v, i, str(round(v)), color='black', ha='left', va='center')

plt.title('top ten profitable category ',fontsize=30)
plt.ylabel('Category',size=25)
plt.xlabel('Revenue(in milion)',size=25)
plt.show()

### Revenue by Month

In [ ]:
plt.figure(figsize=(15,10))

ax = sns.barplot(x='Month',y='Revenue', data=Sales_months)

for p in ax.patches:
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ratio = y / sum(Sales_months['Revenue'])  # Calculate the ratio
    ax.text(x, y, f'{ratio:.2%}', ha='center', va='bottom')

plt.title('Revenue by Month ',fontsize=30)
plt.ylabel('Revenue(in milion)',size=25)
plt.xlabel('Months',size=25)
plt.show()

- The months of June to August are almost the same, from the month of September there is a serious increase that continues until the month of November which is the peak.

- See also linear graph.

### Revenue by Months

In [ ]:
plt.figure(figsize=(15, 5))
sns.lineplot(data=Sales_months, x="Month", y="Revenue")
plt.title("Revenue by Months",size=25)
plt.show()

### Revenue by price range

In [ ]:
price_range_data

In [ ]:
labels = ['elite', 'high', 'high medium', 'low', 'medium']
revenue = [936890.48, 2239804.13, 2647086.42, 1898839.33, 1624586.84]

# Plot
plt.figure(figsize=(8, 8))
plt.pie(revenue, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Revenue Share by Price Range',size=25)

plt.axis('equal')
plt.show()

**Recommendation:** expand the range of products whose price belongs to the "high medium" category.

## Returned Products

In [ ]:
returned['Quantity'] *= -1
returned['Revenue'] = returned['Quantity'] * returned['UnitPrice']

#the most ten returned product
returned.groupby('Description')['InvoiceNo'].count().reset_index().sort_values(by='InvoiceNo',ascending=False).head(12)

"REGENCY CAKESTAND 3 TIER" it's the product that is much addicted so it makes sense that it will also have more cancellation.

In [ ]:
print('the share of returned from all dataset is: {:.2%}'.format(len(returned)/len(data)))

In [ ]:
print('the sum of money back for returned product is: {:.2f}'.format(returned.Revenue.sum()))

In [ ]:
print('the sum of the quntity that returned: {}'.format(returned.Quantity.sum()))
print('the share from all quantity that was sale: {:.2%}'.format(returned.Quantity.sum()/data.Quantity.sum()))

## Checking hypotheses:

- Differences in avarge unit price between price groups "low" and "medium".

- Differences in avarge unit price between top 2 ordered product.

im going to use in The stats.f_oneway() test, also known as the one-way ANOVA (Analysis of Variance) test, is a statistical test used to determine whether there are significant differences between the means of two or more groups. It assesses whether the variability between group means is greater than the variability within the groups.
its will help me to check those tow hypotheses.

### preparing the data for the tests

In [ ]:
avg_rev_group = data.groupby('Price_Range')['UnitPrice'].mean().reset_index()
avg_rev_group

In [ ]:
medium = data[data['Price_Range']=='medium']['UnitPrice']
low = data[data['Price_Range']=='low']['UnitPrice']

product1 = data[data['Description']=='white hanging heart t-light holder']['UnitPrice']
product2 = data[data['Description']=='regency cakestand 3 tier']['UnitPrice']

### conducting the test

- Differences in average unit price between price groups "low" and "medium".

**Ho : There is no significant difference between the average of price group "low" and "medium".**

**H1 : There is significant difference between the average of price group "low" and "medium".**

In [ ]:
t_statistic, p_value = stats.ttest_ind(low, medium)

print(len(medium), medium.mean())
print(len(low), low.mean())
print('stat=%.3f, p=%.3f' % (t_statistic, p_value))
if p_value < 0.05:
    print('There is no significant difference between the average of price group "low" and "medium".')
else:
    print('There is significant difference between the average of price group "low" and "medium".')
    


we can't reject the null hypothese since the differnces between the avarges are too small. 

- Differences in avarge unit price between top 2 ordered product.

**Ho: There is no significant difference between the average price of the top 2 ordered product.**

**Ho: There is significant difference between the average price of the top 2 ordered product.**


In [ ]:
t_statistic, p_value = stats.ttest_ind(product1, product2)

print(len(product1), product1.mean())
print(len(product2), product2.mean())
print('f_statistic = %.3f , p = %.3f' % (t_statistic, p_value))
if p_value < 0.05:
    print("There are significant differences in average income between price groups.")
else:
    print("No significant differences in average income between price groups.")

according this we reject the null hypothese since there is significant differences in average income between those two products.

# recomnendation

- expand the range of products related to home accessories, home kitchen, food packaging solutions.

- expand the range of products whose price belongs to the "high medium" category.

## Dashboard

link to the Dashboard: https://public.tableau.com/shared/N4QQYWH63?:display_count=n&:origin=viz_share_link

In [ ]:
#loading the new data after all the change i made to new csv file 

#data.to_csv('data.csv',index=False)

## Presentaion
https://drive.google.com/file/d/1DvsuKKA6KwWq1QvUgVONyAlAyibBTvx3/view?usp=sharing